In [1]:
import tensorflow as tf
import pathlib
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from custom_fn import specificity,recall

2023-11-25 00:10:04.562395: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-25 00:10:04.585187: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 00:10:04.585212: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 00:10:04.585229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 00:10:04.589664: I tensorflow/core/platform/cpu_feature_g

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=5300)]
    )

2023-11-25 00:10:05.622282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 00:10:05.625323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 00:10:05.625434: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
data_dir = pathlib.Path('./Datasets/lung/').with_suffix('')
train_ds,val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="both",
  label_mode="binary",
  seed=123,
  image_size=(512, 512),
  batch_size=32)

Found 977 files belonging to 2 classes.
Using 684 files for training.


Using 293 files for validation.


2023-11-25 00:10:05.664424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 00:10:05.664557: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 00:10:05.664627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
model=Sequential()
model.add(Conv2D(filters= 8,kernel_size=(4,4),input_shape=(512,512,3),padding='same'))
model.add(Conv2D(filters= 16,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters= 32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(units=16,activation='relu'))
model.add(Dense(units=8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 8)       392       
                                                                 
 conv2d_1 (Conv2D)           (None, 510, 510, 16)      1168      
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 16)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 253, 253, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 64)      1

In [5]:
model.compile(
  optimizer='adam',
  loss=keras.losses.BinaryCrossentropy(),
  metrics=['accuracy',specificity,recall])

In [6]:
from keras.callbacks import EarlyStopping
es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=6,verbose=1)
cd=[es]

In [7]:
model.fit(train_ds,
  validation_data=val_ds,   epochs=30,
  use_multiprocessing=True, callbacks=cd)

model.save('bestmodel60_40.keras')

Epoch 1/30


2023-11-25 00:10:07.276804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-25 00:10:09.606105: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fae643ba0d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-25 00:10:09.606129: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2023-11-25 00:10:09.609229: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-25 00:10:09.685129: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


22/22 [==============================] - 14s 349ms/step - loss: 18.6003 - accuracy: 0.4518 - specificity: 0.2574 - recall: 0.7429 - val_loss: 0.6932 - val_accuracy: 0.4096 - val_specificity: 0.0000e+00 - val_recall: 1.0000
Epoch 2/30
22/22 [==============================] - 5s 238ms/step - loss: 0.6930 - accuracy: 0.5673 - specificity: 0.9545 - recall: 0.0455 - val_loss: 0.6926 - val_accuracy: 0.5904 - val_specificity: 1.0000 - val_recall: 0.0000e+00
Epoch 3/30
10/22 [============>.................] - ETA: 2s - loss: 0.6926 - accuracy: 0.5688 - specificity: 1.0000 - recall: 0.0000e+00